In [88]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split 
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

In [5]:
PATH = 'D:/resized_animal_10/content/processed_data'
labels = open('labels.txt', 'w')
mapping = {'cane':'0', 'cavallo':'1', 'farfalla':'2', 'gatto':'3', 'pecora':'4'}


i=0
for folder in sorted(os.listdir(PATH)):
    for file in sorted(os.listdir(os.path.join(PATH,folder))):
        ext = os.path.splitext(file)[1]
        copyfile(os.path.join(PATH, folder, file), os.path.join('data', str(i)+ext))
        labels.write(mapping[folder]+'\n')
        i+=1

labels.close()

In [9]:
for folder in ['gatto']:
    i=0
    for file in sorted(os.listdir(os.path.join(PATH,folder))):
        ext = os.path.splitext(file)[1]
        #print(os.path.join(PATH, folder, file))
        os.rename(os.path.join(PATH, folder, file), os.path.join(PATH, folder, str(i)+ext))
        i+=1

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:/resized_animal_10/content/processed_data\\gatto\\100.jpeg' -> 'D:/resized_animal_10/content/processed_data\\gatto\\2.jpeg'

In [ ]:
for folder in sorted(os.listdir(PATH)):
    print(len(os.listdir(os.path.join(PATH,folder))))

4863
2623
2112
1668
1820


In [89]:
# Model 
class Image_Classification_DeepCNN(nn.Module):
    def __init__(self, model=None):
        super(Image_Classification_DeepCNN, self).__init__()

        if model=='vgg16':
            self.model = models.vgg16(pretrained=True)
            self.model.classifier = nn.Sequential(nn.Linear(25088, 4096), nn.Tanh(), nn.Linear(4096, 1024), nn.Tanh(), nn.Linear(1024, 128), nn.Tanh(), nn.Linear(128, 5))
            
        if model=='googlenet':
            self.model = models.googlenet(pretrained=True)
            self.model.fc = nn.Sequential(nn.Linear(1024, 256), nn.Tanh(), nn.Linear(256, 32), nn.Tanh(), nn.Linear(32, 5))

    def forward(self, x):
        x = self.model(x)
        return x


def test():
    model = Image_Classification_DeepCNN(model='vgg16')
    input = torch.randn(1, 3, 224, 224)
    out = model(input)

    print(model)
    print(out)
    print(torch.argmax(out).float())    

#test()

In [98]:
# DATASET
transform = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])


class Image_Dataset(Dataset):
    def __init__(self, image_dir, label_dir, transforms=transform):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.images = sorted(os.listdir(self.image_dir))
        self.labels = np.loadtxt(self.label_dir)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        #img = np.array(Image.open(img_path), dtype=np.int32)
        img = Image.open(img_path)
        label = torch.tensor(self.labels[index])

        if self.transforms:
            pre_processing = self.transforms
            img = pre_processing(img)

        return (img, label)


def test():
    test_dataset = Image_Dataset(image_dir='dataset/data', label_dir='dataset/labels.txt', transforms=transform)
    (test_image, test_label) = test_dataset[5342]
    print(test_image.shape, type(test_image), test_image.max(), test_image.min(), test_label)
    
    plt.figure()
    plt.imshow(test_image.numpy().transpose(1,2,0))
    plt.axis('off')
    plt.show()

#test()

In [101]:
# TRAIN
print('Enter - 0 for vgg16 OR 1 for googlenet')
model_map = {'0':'vgg16', '1':'googlenet'}
model_type = str(input())
model_type = model_map[model_type]
print(f'Model : {model_type}')

image_dir = 'dataset/data'
label_dir = 'dataset/labels.txt'
device = 'cpu'
batch_size = 1
learning_rate = 1e-3
epochs = 200


dataset = Image_Dataset(image_dir=image_dir, label_dir=label_dir)
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)


model = Image_Classification_DeepCNN(model=model_type).to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

avg_train_losses = list()
avg_val_losses = list()
for epoch in range(epochs):
    count = 0
    avg_train_loss = 0

    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        count += 1

        image = image.to(device=device)
        label = label.to(device=device)

        out = model(image)
        loss = criterion(out, label.long())

        avg_train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        
    avg_train_loss = avg_train_loss/count
    avg_train_losses.append(avg_train_loss)


    count = 0
    avg_val_loss = 0
    model.eval()
    for batch_idx, (image, label) in enumerate(val_loader):
        with torch.no_grad():
            count += 1

            image = image.to(device=device)
            label = label.to(device=device)

            out = model(image)
            loss = criterion(out, label.long())

            avg_val_loss += loss.item()
        
    
    avg_val_loss = avg_val_loss/count
    avg_val_losses.append(avg_val_loss)
    print(f'Epochs:{epoch+1}, Average Train Loss:{avg_train_loss}, Average Validation Loss:{avg_val_loss}')

    if ((epoch+1)%10==0):
        torch.save(model.state_dict, f'model_weights_{model_type}.pth')


# Plot the loss functions
plt.figure()
plt.plot(list(range(1,epochs+1)), avg_train_losses, 'b', label="Train Loss")
plt.plot(list(range(1,epochs+1)), avg_val_losses, 'r', label="Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.title("Average Loss v/s Epoch")
plt.legend()
plt.savefig(f"plots/vgg16_loss.png")
plt.show()

Enter - 0 for vgg16 OR 1 for googlenet
Model : vgg16


KeyboardInterrupt: 

In [ ]:
# EVAL 
train_loader = DataLoader(dataset=train_dataset)
val_loader = DataLoader(dataset=val_dataset)

model = Image_Classification_DeepCNN(model=model_type).to(device=device)
model.load_state_dict(torch.load(f'model_weights_{model_type}.pth'))

def accuracy(loader, model):
    y_pred = []
    y = []

    print("------------------------------------------------------")

    num_correct_labels = 0
    num_labels = 0

    with torch.no_grad():
        for image, label in loader:
            model.eval()
            image = image.to(device=device)
            label = label.to(device=device)

            out = model(image)
            preds = torch.argmax(out)
            y_pred.append(preds.item())
            y.append(int(label.item()))
            num_correct_labels += (preds==label)
            num_labels += 1

        print(f"Accuracy of the model is {float(num_correct_labels/num_labels)*100:.2f} %")
        print("--------------------------------------------------------------------------------")
        print('')

    model.train()

    return (y_pred, y)

    


print("Checking accuracy on train data..")
y_pred_train, y_train = accuracy(train_loader, model)

print("Checking accuracy on validation data..")
y_pred_val, y_val = accuracy(val_loader, model)